In [1]:
from time import time
from concurrent.futures import ThreadPoolExecutor

import PIL
from PIL import Image
import numpy as np
import pandas as pd
import pickle as pk
import os
from PIL import Image
from keras.callbacks import EarlyStopping, ModelCheckpoint

from tqdm import tqdm
import numpy
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

Using Theano backend.


Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 950M (0000:01:00.0)


In [2]:
def process_files_async(files_path, _func):
    imgs_array = []
    targets = []

    with ThreadPoolExecutor() as pool:
        for img_array, target in tqdm(pool.map(_func, files_path), total=len(files_path)):
            imgs_array.append(img_array)
            targets.append(target)
    return imgs_array, targets

def process_img_train(img_path):
    img_array = img_to_matrix(img_path)
    target = os.path.basename(img_path).split('_')[0]
    return img_array, target

def process_img_test(img_path):
    img_array = img_to_matrix(img_path)
    id = img_path
    return id, img_array

def img_to_matrix(img_path):
    img = Image.open(img_path)
    img = norm_image(img)
    img = resize_image(img, 32)
    img_array = np.asarray(img.convert("RGB"), dtype=np.float32) / 255
    return img_array

def norm_image(img):
    """
    Normalize PIL image

    Normalizes luminance to (mean,std)=(0,1), and applies a [1%, 99%] contrast stretch
    """
    img_y, img_b, img_r = img.convert('YCbCr').split()

    img_y_np = np.asarray(img_y).astype(float)

    img_y_np /= 255
    img_y_np -= img_y_np.mean()
    img_y_np /= img_y_np.std()
    scale = np.max([np.abs(np.percentile(img_y_np, 1.0)),
                    np.abs(np.percentile(img_y_np, 99.0))])
    img_y_np = img_y_np / scale
    img_y_np = np.clip(img_y_np, -1.0, 1.0)
    img_y_np = (img_y_np + 1.0) / 2.0

    img_y_np = (img_y_np * 255 + 0.5).astype(np.uint8)

    img_y = Image.fromarray(img_y_np)

    img_ybr = Image.merge('YCbCr', (img_y, img_b, img_r))

    img_nrm = img_ybr.convert('RGB')

    return img_nrm


def resize_image(img, size):
    """
    Resize PIL image

    Resizes image to be square with sidelength size. Pads with black if needed.
    """
    # Resize
    n_x, n_y = img.size
    if n_y > n_x:
        n_y_new = size
        n_x_new = int(size * n_x / n_y + 0.5)
    else:
        n_x_new = size
        n_y_new = int(size * n_y / n_x + 0.5)

    img_res = img.resize((n_x_new, n_y_new), resample=PIL.Image.BICUBIC)

    # Pad the borders to create a square image
    img_pad = Image.new('RGB', (size, size), (128, 128, 128))
    ulc = ((size - n_x_new) // 2, (size - n_y_new) // 2)
    img_pad.paste(img_res, ulc)

    return img_pad


def get_files_path(dir):
    files  = []
    for f in os.listdir(dir):
        if os.path.isdir(os.path.join(dir,f)):
            files.extend(get_files_path(os.path.join(dir,f)))
        elif '.jpg' in f:
            files.append(os.path.join(dir,f))
    return files


def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.

    E.g. for use with categorical_crossentropy.

    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.

    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y)
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y-1] = 1
    return categorical

In [42]:
dir_input = os.path.abspath('./input/')
dir_train = os.path.join(dir_input, 'train')
dir_test = os.path.join(dir_input, 'test')
paths_train = get_files_path(dir_train)
paths_test = get_files_path(dir_test)
x_train, y_train = process_files_async(paths_train, process_img_train)
id_test, x_test = process_files_async(paths_test, process_img_test)



  0%|          | 0/22897 [00:00<?, ?it/s]

  0%|          | 41/22897 [00:00<01:28, 258.29it/s]



  0%|          | 81/22897 [00:00<01:29, 255.26it/s]



  1%|          | 126/22897 [00:00<01:18, 290.19it/s]



  1%|          | 186/22897 [00:00<01:06, 339.71it/s]

...

 95%|█████████▌| 7356/7710 [00:33<00:01, 191.93it/s]

 96%|█████████▌| 7395/7710 [00:33<00:01, 198.05it/s]

 96%|█████████▋| 7429/7710 [00:33<00:01, 217.28it/s]

 97%|█████████▋| 7461/7710 [00:35<00:04, 60.35it/s] 

100%|██████████| 7710/7710 [00:35<00:00, 218.76it/s]

In [ ]:
numpy.random.seed(42)
DENSE_SIZE = 512

batch_size = 32
nb_classes = 257
nb_epoch = 25
img_rows, img_cols = 32, 32
img_channels = 3

STAMP = 'batch{}_size{}DENSE_SIZE{}'.format(batch_size, img_rows, DENSE_SIZE)
X_train, X_valid, Y_train, Y_valid = train_test_split(np.asarray(x_train), np.asarray(y_train), stratify=np.asarray(y_train), test_size=0.2)


X_train=np.asarray(X_train)
Y_train=np.asarray(Y_train)

Y_train = to_categorical(Y_train, nb_classes)
Y_valid = to_categorical(Y_valid, nb_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(32, 32, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(DENSE_SIZE, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='accuracy', patience=2)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
#todo fix callbacks crash app
hist = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=nb_epoch,
              # callbacks=[early_stopping, model_checkpoint],
              validation_data=(X_valid, Y_valid))

# model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_acc'])
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_valid, Y_valid, verbose=0)
print("Acc validation: %.2f%%" % (bst_val_score*100))
predict = model.predict(x_test)
pd.DataFrame({'id':id_test, 'class':predict}).to_csv('submission.csv', index=False)